https://colab.research.google.com/drive/1uMKT1vX88BZo21Ph0Gc-cDNpm7SkiK1G?authuser=0#scrollTo=LDZI4EpWGRz6

In [2]:
!pip install lightautoml

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.3 MB/s eta 0:00:00
  

In [1]:
import pandas as pd
import numpy as np
from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.report.report_deco import ReportDeco
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')



/usr/local/lib/python3.12/dist-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


In [2]:
def load_and_preprocess_data(file_path):
    """Загрузка и предобработка данных аналогично NN_deepseek"""
    print("Загрузка данных...")
    data = pd.read_csv(file_path, sep=";", index_col=False)

    # Создание целевой переменной: 1 если N_claims_year > 1, иначе 0
    data['claim_prob'] = (data['N_claims_year'] > 1).astype(int)

    # Удаление ненужных столбцов
    cols_to_drop = ['ID', 'Date_start_contract', 'Date_last_renewal', 'Date_next_renewal',
                    'Date_lapse', 'N_claims_year', 'Cost_claims_year', 'N_claims_history',
                    'R_Claims_history']
    data = data.drop(columns=cols_to_drop)

    return data



In [3]:
def process_dates_and_features(data):
    """Обработка дат и создание новых признаков"""
    print("Обработка дат и признаков...")

    # Обработка датовых колонок
    date_columns = ['Date_birth', 'Date_driving_licence']
    existing_date_columns = [col for col in date_columns if col in data.columns]

    if not existing_date_columns:
        print("Предупреждение: Датовые колонки не найдены в DataFrame.")
    else:
        for col in existing_date_columns:
            data[col] = pd.to_datetime(data[col], format='%d/%m/%Y', errors='coerce')

        # Извлечение числовых признаков из дат
        reference_date = pd.to_datetime('2019-12-31')

        if 'Date_birth' in data.columns:
            data['Age'] = (reference_date - data['Date_birth']).dt.days // 365

        if 'Date_driving_licence' in data.columns:
            data['Driving_experience'] = (reference_date - data['Date_driving_licence']).dt.days // 365

    # Удаление исходных датовых колонок
    data = data.drop(columns=date_columns, errors='ignore')

    # Обработка категориальных переменных
    categorical_cols = ['Type_fuel']
    for col in categorical_cols:
        if col in data.columns:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))

    # Убедиться, что все данные числовые
    data = data.apply(pd.to_numeric, errors='coerce')

    # Обработка пропущенных значений
    imputer = SimpleImputer(strategy='median')
    data_imputed = imputer.fit_transform(data)
    data = pd.DataFrame(data_imputed, columns=data.columns)

    return data



In [4]:
def train_automl_model(X_train, y_train, X_test, y_test):
    """Обучение AutoML модели для бинарной классификации"""
    print("Настройка AutoML...")

    # Объединение признаков и целевой переменной для LightAutoML
    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    # Настройка AutoML для бинарной классификации
    TARGET_NAME = 'claim_prob'
    task = Task('binary')

    automl = TabularAutoML(
        task=task,
        timeout=3600,  # Максимальное время работы модели (секунды)
        cpu_limit=4,   # Количество CPU
        memory_limit=16  # Ограничение памяти (ГБ)
    )

    # Обертка для генерации отчетов
    automl_with_report = ReportDeco(output_path='AutoML_classification_report.html')(automl)

    # Определяем роли переменных
    roles = {'target': TARGET_NAME}

    print("Обучение AutoML модели...")
    # Обучение модели
    oof_pred = automl_with_report.fit_predict(
        train_data=train_data,
        roles=roles,
        verbose=1
    )

    return automl, test_data, train_data




In [5]:
def evaluate_model(automl, test_data, train_data, y_test, y_train):
    """Оценка модели и вывод результатов"""
    print("Оценка модели...")

    # Предсказание на тестовых данных
    test_pred = automl.predict(test_data)
    test_proba = test_pred.data[:, 0]  # Вероятности класса 1

    # Предсказание на тренировочных данных
    train_pred = automl.predict(train_data)
    train_proba = train_pred.data[:, 0]

    # Метрики
    roc_auc_test = roc_auc_score(y_test, test_proba)
    mae_test = mean_absolute_error(y_test, test_proba)
    roc_auc_train = roc_auc_score(y_train, train_proba)

    print("\n" + "="*50)
    print("РЕЗУЛЬТАТЫ AUTOML ДЛЯ БИНАРНОЙ КЛАССИФИКАЦИИ")
    print("="*50)
    print(f"ROC-AUC на тестовых данных: {roc_auc_test:.4f}")
    print(f"MAE на тестовых данных: {mae_test:.4f}")
    print(f"ROC-AUC на тренировочных данных: {roc_auc_train:.4f}")

    # Анализ важности признаков
    try:
        feature_scores = automl.get_feature_scores()
        feature_scores = feature_scores.sort_values('importance', ascending=False)

        print("\nТоп-10 важных признаков:")
        print("-" * 30)
        for i, (feature, importance) in enumerate(feature_scores.head(10)['importance'].items(), 1):
            print(f"{i:2d}. {feature:<25} {importance:.4f}")
    except Exception as e:
        print(f"\nНе удалось получить важность признаков: {e}")

    return test_proba, train_proba



In [7]:
def main():
    """Основная функция"""
    file_path = 'sample_data/vehicle_ins_data_1.csv'

    try:
        # 1. Загрузка и предобработка данных
        data = load_and_preprocess_data(file_path)

        # 2. Обработка признаков
        data = process_dates_and_features(data)

        # 3. Разделение на обучающую и тестовую выборки
        print("Разделение данных на train/test...")
        X = data.drop(columns=['claim_prob'])
        y = data['claim_prob']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        print(f"Размеры данных: Train {X_train.shape}, Test {X_test.shape}")

        # 4. Обучение AutoML модели
        automl, test_data, train_data = train_automl_model(X_train, y_train, X_test, y_test)

        # 5. Оценка модели
        test_proba, train_proba = evaluate_model(automl, test_data, train_data, y_test, y_train)

        print("\nAutoML обучение завершено успешно!")
        print("Отчет сохранен в файл: AutoML_classification_report.html")

    except Exception as e:
        print(f"Произошла ошибка: {e}")
        raise

if __name__ == "__main__":
    main()

Загрузка данных...
Обработка дат и признаков...
Разделение данных на train/test...
Размеры данных: Train (84444, 21), Test (21111, 21)
Настройка AutoML...
Обучение AutoML модели...
[08:49:52] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[08:49:52] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


[08:49:52] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



...

[09:10:46] Automl preset training completed in 1253.72 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 1253.72 seconds



[09:10:46] Model description:
Final prediction for new objects (level 0) = 
	 0.10234 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.25620 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.52619 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.11527 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.10234 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.25620 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.52619 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.11527 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



Оценка модели...

РЕЗУЛЬТАТЫ AUTOML ДЛЯ БИНАРНОЙ КЛАССИФИКАЦИИ
ROC-AUC на тестовых данных: 0.7561
MAE на тестовых данных: 0.1401
ROC-AUC на тренировочных данных: 0.9170

Не удалось получить важность признаков: 'importance'

AutoML обучение завершено успешно!
Отчет сохранен в файл: AutoML_classification_report.html
